# Interpretable ML over an Extended Version of  the French Royalty KG

This an example on how **InterpretME** can be used to interpret the prediction and trace back a particular target entity. The KG of the *French Royalty Benchmark* is a fully curated subset of DBpedia; for each person we added the class `dbo:Person` as well as different properties like the number of children or predecessors, and further triple related counts. Here, the predictive task is a binary classification to predict whether a person has a spouse. The statistics of the *French Royalty KG* are presented in the following:

| #triples | #entities | #predicates | #objects | #triples / #entities |
| :-: | :-: | :-: | :-: | :-: |
| 31,599 | 3,439 | 133 | 4,390 | 9.18 |

## Let's Start with Essentials for this Tutorial Notebook

In [1]:
%%capture
pip install /Users/a.arnous/workspace/InterpretME


<div class="alert alert-info"><b>Note:</b> You may need to restart the kernel to use the updated packages!</div>

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The mechanism of InterpretME is as follows:<br>
<ol>
  <li>SHACL validation</li>
  <li>Data preprocessing</li>
  <li>Training of the predictive model</li>
  <li>Understanding the results of interpretability models (e.g., LIME)</li>
  <li>Semantify collected metadata</li>
  <li>Uploading semantified metadata into Virtuoso</li>
  <li>Querying the InterpretME KG and input KG to trace back all properties of a particular entity</li>
</ol>

<div class="alert alert-info"><b>Note:</b> If you don't have Docker (and docker-compose) installed. Please, follow the instructions according to configuration of your machine.

<ul>
        <li><b>Linux:</b> <a href="https://docs.docker.com/desktop/install/linux-install/" target="_blank">https://docs.docker.com/desktop/install/linux-install/</a></li>
        <li><b>Windows:</b> <a href="https://docs.docker.com/docker-for-windows/install/" target="_blank">https://docs.docker.com/docker-for-windows/install/</a></li>
        <li><b>Mac:</b> <a href="https://docs.docker.com/docker-for-mac/install/" target="_blank">https://docs.docker.com/docker-for-mac/install/</a></li>
</ul>
</div>

To get started with the **KG**, you need to start the containers:

In [3]:
!docker-compose up -d

interpretmekg is up-to-date
frenchroyalty is up-to-date
interpretme is up-to-date


Importing required modules from **InterpretME** library:

* `pipeline()`: Run the predictive tasks and interpretation tools (e.g., LIME).
* `plots.sampling()`: Generates plot of the target class distribution.
* `plots.feature_importance()`: Creates bar plot of important features.
* `plots.decision_trees()`: Generates trees of predictions made by predictive model.
* `plots.constraints_decision_trees()`: Trees are incorporated with SHACL validation results.
* `federated()`: Query the *InterpretME KG* and the input KG to trace back all properties of a target entity.


In [4]:
from InterpretME import *

**InterpretME** takes a JSON file as input (i.e., *URL of the input KG, features’ definition, target definition, SHACL constraints, sampling strategy, class definition*); a `SPARQL query` is generated based on the feature definition given by the user and the query is used to retrieve the application domain data from the input KG.

Given the input **KG** that integrates the features’ and class target definitions about French Royalty; and their SHACL constraints. The features’ definition is classified into independent and dependent variables; later used in the predictive modeling pipeline. The features can be defined in the following format:

```JSON
{
    "path_to_data": "dataset/tlos_v1.csv",
    "Type": "Patient",
    "Index_var": "index",
    "Independent_variable": ["sex", "age", "an_parenchymal_opacification", "an_cardiac_silhouette_enlargement", "an_pleural_effusion"],
    "Dependent_variable": ["event"],
    "classes": {
      "Dead": "0",
      "Alive": "1"
    },
    "sampling_strategy": "undersampling",
    "number_important_features": 6,
    "cross_validation_folds": 5,
    "test_split": 0.3,
    "model": "Random Forest",
    "min_max_depth": 4,
    "max_max_depth": 6
  }
```

<div class="alert alert-info"><b>Note:</b> As of v1.2.0, InterpretME is also able to work with CSV and JSON datasets. See `example_csv_french_royalty.json` for an example configuration for datasets.</div>

The purpose of `pipeline()` is to assemble several components of **InterpretME** that can be evaluated together while setting different parameters. First, it starts with evaluating the SHACL constraints over the nodes of input KGs and generates a validation report per target entity. This report shows if a particular entity validates/invalidate the constraints defined by the user. 

The *data preprocessing* step includes transforming the data extracted from the input KG into a form that can be used to train the predictive pipeline. To avoid imbalance, the sampling strategy defined by the user is deployed. To handle categorical values from data, one-hot encoding is utilized.

The *predictive model building* step can be achieved based on user preferences. Given the French Royalty preprocessed data, automated tools are utilized for models (e.g., *Ensemble Learning*) and to optimize the hyperparameter selection for predictive tasks. 

Here, the automated predictive model can perform stratified shuffle split cross-validation with *Random Forest*, *Adaboost Classifier*, or *Gradient Boosting Classifier* and identify the relevant features; they are used to train a *Decision Tree* classifier to predict and visualize the outcomes.

In this step, the metadata collected are the features' definition, trained model, hyperparameters, predictions, precision, recall, classification report, as well as confusion matrix files generated from the trained predictive model. The metadata are later used in the creation of InterpretME KG.

The current version of InterpretME uses *LIME* [1] to have local interpretations of the target entities.
*LIME* also identifies the top-10 relevant features for the target entity and assigns weights. The results allow the user to understand the quality of the predictive model.

The RDF mapping language (RML) is used to define mappings for the metadata collected from the predictive pipeline in order to integrate them into the **InterpretME KG**.
The RML mappings are used by the SDM-RDFizer [2], an efficient RML engine for creating knowledge graphs, to semantify the metadata. InterpretME relies on **FAIR** principles for defining the *InterpretME ontology* by extending *ML schema*; it is available on [VoCol](http://ontology.tib.eu/InterpretME/).

The generated RDF data will be uploaded to an instance of *Virtuoso*.

In [5]:
results = pipeline(path_config='./example_cvs_tlos_v1.json', lime_results='./output/lime', survshap_results='./output/survshap', server_url='http://localhost:8891/', username='dba', password='dba', survival=1)
   

InterpretME Pipeline:   0%|          | 0/5 [00:00<?, ?task/s]

TypeError: exepected pandas.DataFrame, but got <class 'numpy.ndarray'>

### Exploration of Predictive Results via Decision Trees

To understand the trained predictive model's decisions, an automated model is deployed and a visualization of the prediction is performed via *Decision Trees*. Here, the user can also visualize SHACL constraints with decision trees which shows entities violating the constraints/validating constraints. 

The user can provide the path where to store the output plots, like sampling strategy (target class distribution), feature importance, and decision trees.

In [ ]:
# show SVG in Jupyter notebook
from IPython.display import SVG, display
import os
def show_svg(rel_path):
    if type(rel_path) == list:
        for path in rel_path:
            show_svg(path)
    else:
        display(SVG(url='file://' + os.getcwd() + '/' + rel_path))

#### Sampling

In [ ]:
plots.sampling(results=results, path='./output/')

#### Feature Importance

In [ ]:
plots.feature_importance(results=results, path='./output/')

#### Decision Trees

In [ ]:
show_svg(plots.decision_trees(results=results, path='./output/'))

#### Decision Trees with Constraint Validation

In [ ]:
show_svg(plots.constraints_decision_trees(results=results, path='./output/', constraint_num=[3]))

### Federated Query Processing

After uploading the semantified results to the InterpretME KG, [DeTrusty](https://github.com/SDM-TIB/DeTrusty) [3] is utilized to answer the user's questions via SPARQL queries over **both** the input KG and the *InterpretME KG*. In `./queries/french_royalty` you can find templates for answering the following questions:
1. Which is the target entity interpreted by LIME?
2. How does ***feature*** contribute to the classification of this entity in class ***class***
3. Which other features are relevant for this classification?
4. Does this target entity satisfy the domain integrity constraints?
5. What are the main characteristics of the target entity?

In [ ]:
from InterpretME.federated_query_engine import configuration, federated

In [ ]:
input_query = """
SELECT DISTINCT ?sourceEntity ?InterpretableTool ?feature ?value ?targetClass ?probability
WHERE {
    SERVICE <http://localhost:8891/sparql> {
        FILTER( ?LIMEentity=<http://interpretme.org/entity/Louis_XIV> )
        ?entity a <http://interpretme.org/vocab/TargetEntity> .
        ?entity <http://www.w3.org/2002/07/owl#sameAs> ?sourceEntity .
        ?entity <http://interpretme.org/vocab/hasEntity> ?LIMEentity .
        ?entity <http://interpretme.org/vocab/hasInterpretedFeature> ?interpretedFeature .
        ?interpretedFeature <http://interpretme.org/vocab/hasFeatureWeight> ?featureWeight .
        ?interpretedFeature <http://www.w3.org/ns/prov#hasGeneratedBy> ?InterpretableTool .
        ?entity <http://interpretme.org/vocab/hasEntityClassProbability> ?classProb .
        ?classProb <http://interpretme.org/vocab/hasPredictionProbability> ?probability .
        ?classProb <http://interpretme.org/vocab/hasClass> ?targetClass .
        ?featureWeight <http://interpretme.org/vocab/hasFeature> ?feature .
        ?featureWeight <http://interpretme.org/vocab/hasWeight> ?value .
    }
}
"""


In [ ]:
interpretme_endpoint = 'http://localhost:8891/sparql'
input_endpoint = 'http://localhost:8892/sparql'

In [ ]:
config = configuration(interpretme_endpoint, input_endpoint)
query_answer = federated(input_query, config)
query_answer

## Clean up:

In [ ]:
!docker-compose down -v

------------
## References

[1] Marco Ribeiro, Sameer Singh, and Carlos Guestrin. "Why Should I Trust You?": Explaining the Predictions of Any Classifier. In: Proceedings of the 22nd ACM SIGKDD International Conference on Knowledge Discovery and Data Mining (KDD '16). ACM. 2016. DOI: [10.1145/2939672.2939778](https://dl.acm.org/doi/10.1145/2939672.2939778).

[2] E. Iglesias, S. Jozashoori, D. Chaves-Fraga, D. Collarana and M.-E. Vidal. SDM-RDFizer: An RML Interpreter for the Efficient Creation of RDF Knowledge Graphs. In: CIKM ’20:Proceedings of the 29th ACM International Conference on Information & Knowledge Management, ACM, New York, NY,USA, 2020. DOI: [10.1145/3340531.3412881](https://dl.acm.org/doi/pdf/10.1145/3340531.3412881).

[3] P.D. Rohde, M. Bechara, and Avellino. DeTrusty v0.12.2, June 2023. DOI: [10.5281/zenodo.8063472](https://doi.org/10.5281/zenodo.8063472).